In [107]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.{StringIndexer,HashingTF,IDF,RegexTokenizer,StopWordsRemover,ChiSqSelector}
import org.apache.spark.ml.classification.{LinearSVC, OneVsRest}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.Normalizer
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.util.DefaultParamsWritable
import scala.io.Source.fromFile

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.{StringIndexer, HashingTF, IDF, RegexTokenizer, StopWordsRemover, ChiSqSelector}
import org.apache.spark.ml.classification.{LinearSVC, OneVsRest}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.Normalizer
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.util.DefaultParamsWritable
import scala.io.Source.fromFile


In [1]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder
.appName("SVM Text Classification")
.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_0598
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_0598)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
sc: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4c35d2ad


In [3]:
val path_to_stopwords = "../data/stopwords.txt"

val k = 2000
val split_pattern = "[^a-zA-Z<>^|]+"

path_to_stopwords: String = ../data/stopwords.txt
k: Int = 2000
split_pattern: String = [^a-zA-Z<>^|]+


In [105]:
val reviewsDF = sc
.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")
.select("category","reviewText")

val stopwords = fromFile(path_to_stopwords).getLines.toArray

reviewsDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- reviewText: string (nullable = true)



reviewsDF: org.apache.spark.sql.DataFrame = [category: string, reviewText: string]
stopwords: Array[String] = Array(a, aa, able, about, above, absorbs, accord, according, accordingly, across, actually, after, afterwards, again, against, ain, album, album, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, app, appear, appreciate, appropriate, are, aren, around, as, aside, ask, asking, associated, at, available, away, awfully, b, baby, bb, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, bibs, bike, book, books, both, brief, bulbs, but, by, c, came, ca...


In [108]:
import org.apache.spark.ml.feature.StopWordsRemover
class CustomStopWordsRemover(stopWordsFile: String) extends StopWordsRemover(stopWordsFile) {
    // load and set custom stop words
    val customStopWords: Array[String] = scala.io.Source.fromFile(stopWordsFile).getLines.toArray
    setStopWords(customStopWords)    
}

class CustomTokenizer extends Tokenizer with DefaultParamsWritable {
  // use splitting pattern from exercise 1
  override protected def createTransformFunc: String => Seq[String] = { input =>
    input.toLowerCase.split(split_pattern).toSeq.filter(_.length > 1)
  }
}


// tokenize
val tokenizer = new RegexTokenizer()
.setInputCol("reviewText")
.setOutputCol("words")
.setPattern(split_pattern)

// remove stopwords
val stopWordsFile = path_to_stopwords
val remover = new StopWordsRemover()
.setInputCol(tokenizer.getOutputCol)
.setOutputCol("filtered")
.setStopWords(stopwords)

// turn words into numerical features
val hashingTF = new HashingTF()
.setInputCol(remover.getOutputCol)
.setOutputCol("rawFeatures")

// scale feature
val idf = new IDF()
.setInputCol(hashingTF.getOutputCol)
.setOutputCol("features")

// encode category to numerical label
val indexer = new StringIndexer()
.setInputCol("category")
.setOutputCol("label")

// ChiSqSelector
val selector = new ChiSqSelector()
.setNumTopFeatures(k)
.setFeaturesCol(idf.getOutputCol)
.setLabelCol(indexer.getOutputCol)
.setOutputCol("selected_features")

val pipeline_feature_extraction = new Pipeline().setStages(Array(tokenizer, remover, hashingTF, idf, indexer, selector))

import org.apache.spark.ml.feature.StopWordsRemover
defined class CustomStopWordsRemover
defined class CustomTokenizer
tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_24b3d5d79e92, minTokenLength=1, gaps=true, pattern=[^a-zA-Z<>^|]+, toLowercase=true
stopWordsFile: String = ../data/stopwords.txt
remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_db93a2ff66b4, numStopWords=596, locale=en_US, caseSensitive=false
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_fd33752c86aa, binary=false, numFeatures=262144
idf: org.apache.spark.ml.feature.IDF = idf_0844db36dcd2
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_9f25dcec4ab9
selector: org.apache.spark.ml.feature.ChiSqSelector = chi...


In [49]:
val Array(train, test) = reviewsDF
.sample(withReplacement = false, fraction = 0.05, seed = 42)
.randomSplit(Array(0.8,0.2), seed=42)

import org.apache.spark.ml.classification.{LinearSVC, OneVsRest}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.DataFrame
train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, reviewText: string]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, reviewText: string]


In [69]:
train.show(5)

+----------------+--------------------+
|        category|          reviewText|
+----------------+--------------------+
|Apps_for_Android|6 inches on a rea...|
|Apps_for_Android|A fun and very ad...|
|Apps_for_Android|Amazing very addi...|
|Apps_for_Android|An enjoyable puzz...|
|Apps_for_Android|App has links to ...|
+----------------+--------------------+
only showing top 5 rows



In [111]:
val normalizer = new Normalizer()
  .setInputCol(selector.getOutputCol)
  .setOutputCol("normalized_features")
  .setP(2.0) // L2 normalization

val svm = new LinearSVC()
.setFeaturesCol(normalizer.getOutputCol)
.setLabelCol(indexer.getOutputCol)

val ovr_svm = new OneVsRest()
.setClassifier(svm)
.setFeaturesCol(normalizer.getOutputCol)
.setLabelCol(indexer.getOutputCol)

val pipeline_train_model = new Pipeline()
.setStages(Array(pipeline_feature_extraction, normalizer, ovr_svm))

normalizer: org.apache.spark.ml.feature.Normalizer = Normalizer: uid=normalizer_021404c2b9e7, p=2.0
svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_592faa63a5e8
ovr_svm: org.apache.spark.ml.classification.OneVsRest = oneVsRest_6d359e69b5de
pipeline_train_model: org.apache.spark.ml.Pipeline = pipeline_4b73dc37023a


In [61]:
// train the model 
val svm_model = pipeline_train_model.fit(train)

svm_model: org.apache.spark.ml.PipelineModel = pipeline_59b053669de1


In [124]:
// predict on test data
val predictions = svm_model.transform(test)

predictions: org.apache.spark.sql.DataFrame = [category: string, reviewText: string ... 9 more fields]


In [67]:
predictions.select("category","label","prediction").show()

+----------------+-----+----------+
|        category|label|prediction|
+----------------+-----+----------+
|Apps_for_Android|  8.0|       9.0|
|Apps_for_Android|  8.0|       3.0|
|Apps_for_Android|  8.0|       8.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|      12.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       6.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       8.0|
|Apps_for_Android|  8.0|       3.0|
|Apps_for_Android|  8.0|       3.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       8.0|
|Apps_for_Android|  8.0|      20.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|      20.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       0.0|
|Apps_for_Android|  8.0|       8.0|
+----------------+-----+----------+
only showing top 20 rows



In [125]:
val evaluator = new MulticlassClassificationEvaluator()
.setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_8f4b42641310, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15


In [126]:
val f1_score_default_model = evaluator.evaluate(predictions)

f1_score_default_model: Double = 0.35558985563198786


In [123]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

val evaluator_f1 = new MulticlassClassificationEvaluator()
.setMetricName("f1")

val paramGrid = new ParamGridBuilder()
.addGrid(svm.maxIter, Array(8,34))
.addGrid(svm.regParam, Array(0.05, 0.13, 0.55))
.addGrid(selector.numTopFeatures, Array(144, 2000))
.addGrid(svm.standardization, Array(false, true))
.build()

val trainValidationSplit = new TrainValidationSplit()
.setEstimator(pipeline_train_model)
.setEvaluator(evaluator_f1)
.setEstimatorParamMaps(paramGrid)

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
evaluator_f1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_d93589f1d259, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linearsvc_592faa63a5e8-maxIter: 8,
	chiSqSelector_e407ba37d885-numTopFeatures: 144,
	linearsvc_592faa63a5e8-regParam: 0.05,
	linearsvc_592faa63a5e8-standardization: false
}, {
	linearsvc_592faa63a5e8-maxIter: 8,
	chiSqSelector_e407ba37d885-numTopFeatures: 144,
	linearsvc_592faa63a5e8-regParam: 0.13,
	linearsvc_592faa63a5e8-standardization: false
}, {
	linearsvc_592faa63a5e8-maxIter: 8,
	chiSqSelector_e407ba37d885-numTopFeatures: 144,
	linearsvc_592faa63a5e8-r...


In [113]:
%%time
val grid_search_model = trainValidationSplit.fit(train)

grid_search_model: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_0c02f78a59ee, bestModel=pipeline_4b73dc37023a, trainRatio=0.75


In [131]:
val grid_search_predicton = grid_search_model.transform(test)
val f1_score_grid_search_model = evaluator.evaluate(grid_search_predicton)

f1_score_default_model, f1_score_grid_search_model

<console>: 5: error: ';' expected but ',' found.